# Model Comparison

Compare neutral densities from several models along the orbit of CHAMP (the RF out-of-sample data set).


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib qt

In [3]:
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import sklearn.metrics as metrics

rf_f = "E:\OneDrive\Phys4501\data\RF_FI_GEO_OOS_CHAMP.hdf5"
dt_f = "D:\data\SatDensities\satdrag_database_DTM.hdf5"
jm_f = "D:\data\SatDensities\satdrag_database_CHAMP_JB2008_MSIS.hdf5"
path_mod = os.path.normpath('d:\\GitHub\\ml_fw\\ml_fw') # assumes current working directory is the ml_fw/Notebooks directory

# add the ml_fw module to Python Path and import what we need
sys.path.append(os.path.dirname(path_mod))

In [4]:
from ml_fw import inspect

In [5]:
rf_dat = pd.read_hdf(rf_f)
dt_dat = pd.read_hdf(dt_f)
jm_dat = pd.read_hdf(jm_f)

In [6]:
rf_dat.columns

Index(['1300_02', '43000_09', '85550_13', '94400_18', 'SYM_H index', 'AE',
       'SatLat', 'cos_SatMagLT', 'sin_SatMagLT', 'cos_SatLon', 'sin_SatLon',
       '400kmDensity', 'DateTime', 'storm', 'storm phase',
       '400kmDensity_pred'],
      dtype='object')

In [7]:
# calculate the residuals and normalize by multiplying by 10^12
rf_dat['rf_resid'] = rf_dat['400kmDensity']-rf_dat['400kmDensity_pred']
# DTM 2020 operation
dt_dat['dt_resid'] = (dt_dat['400kmDensity'] - dt_dat['DTM_400kmDensity']*1000)*(10**12) # convert g/cm^3 to km/m^3
dt_dat['DTM_400kmDensity'] = (dt_dat['DTM_400kmDensity']*1000)*(10**12)

#JB2008
jm_dat['jb_resid'] = (jm_dat['400kmDensity'] - jm_dat['JB2008_400kmDensity'])*(10**12)
jm_dat['JB2008_400kmDensity'] = (jm_dat['JB2008_400kmDensity'])*(10**12)

#MSIS
jm_dat['ms_resid'] = (jm_dat['400kmDensity'] - jm_dat['MSIS_400kmDensity'])*(10**12)
jm_dat['MSIS_400kmDensity'] = (jm_dat['MSIS_400kmDensity'])*(10**12)
       

In [8]:
tol = pd.Timedelta('1 minute')

db_dat = pd.merge_asof(left=rf_dat.set_index("DateTime"),right=dt_dat.set_index('DateTime')[['dt_resid','DTM_400kmDensity']],right_index=True,left_index=True,direction='nearest',tolerance=tol)
db_dat = pd.merge_asof(left=db_dat, right=jm_dat.set_index('DateTime')[['jb_resid','JB2008_400kmDensity','ms_resid','MSIS_400kmDensity']],right_index=True,left_index=True,direction='nearest',tolerance=tol)
db_dat = db_dat.reset_index(drop=False)

# we are only looking at storms so drop the na
db_dat = db_dat.dropna()

In [9]:
db_dat.columns

Index(['DateTime', '1300_02', '43000_09', '85550_13', '94400_18',
       'SYM_H index', 'AE', 'SatLat', 'cos_SatMagLT', 'sin_SatMagLT',
       'cos_SatLon', 'sin_SatLon', '400kmDensity', 'storm', 'storm phase',
       '400kmDensity_pred', 'rf_resid', 'dt_resid', 'DTM_400kmDensity',
       'jb_resid', 'JB2008_400kmDensity', 'ms_resid', 'MSIS_400kmDensity'],
      dtype='object')

In [10]:
# define the x_cols of dat that we want to use for binning
# and their ranges and number of bins
x_dat = ['AE', 'SYM_H index']
x_bin = [20,21]
x_range = [[0,2000], [-200,10]]
# the y_col to derive the stats
y_dat = ['rf_resid','dt_resid','jb_resid','ms_resid']
y_lab = ['RF', 'DTM Op', 'JB2008', 'MSIS']

whisker = 0 # don't want whiskers
showmean = True # plot the means

# box plot colors
cc = ([1,0,0],
      [0.13333333333333333, 0.5450980392156862, 0.13333333333333333],
      [0.27450980392156865, 0.5098039215686274, 0.7058823529411765],
      [1.0, 0.5490196078431373, 0.0,])
bx_a = 0.25 # transparency level (alpha) for box
ln_a = 1.0 # transparency level for lines
ln_w = 2.0 # line width

In [11]:
# plot the data
InteractiveShell.ast_node_interactivity = "last_expr"

In [12]:
# derive the box/whisker data for each resid
fig, ax = plt.subplots(len(x_dat),1, figsize=(6,8))
fig.set_constrained_layout(True)
i=0
for yp, bc, yl in zip(y_dat, cc, y_lab):
    box_dat = inspect.boxplot_vx(db_dat[x_dat],db_dat[yp], whisker=whisker,
                            bins=x_bin,xrange=x_range)

    # lets do the same thing but loop through the returned box_dat
    # to create a box/whisker plot for each returned key (x_dat)
    # loop through the dictionary to plot all box_plots
    
    for (key, value), ap, in zip(box_dat.items(), ax):
        
        plt_box = box_dat[key]
        y_val = plt_box['box_stats']
        x_val = plt_box['x_centre']-plt_box['x_width']/2+i*plt_box['x_width']/len(y_dat)
        x_width = plt_box['x_width']/len(y_dat)
        
        b = ap.bxp(y_val, positions=x_val, widths=x_width, label=yl,
                        patch_artist=True, showmeans=showmean, 
                        shownotches=False, showcaps=False, 
                        boxprops={'ec':bc+[ln_a], 'fc':bc+[bx_a]},
                        medianprops={'c':bc, 'lw':ln_w}, 
                        meanprops={'mec':bc, 'mfc':bc})
        ap.set_xticks(x_val, plt_box['x_centre'].astype(int).astype(str),rotation=45)
        ap.set_ylabel('Residuals')
        ap.set_xlabel(key)

        print(f'{yp}, {key}')
        
    i = i+1

ax[0].legend(bbox_to_anchor=(1., 1),loc='upper right',fontsize=8)
ax[0].get_legend().set_title("Model")

plt.tight_layout()
plt.show()
fig.show()

rf_resid, AE
rf_resid, SYM_H index
dt_resid, AE
dt_resid, SYM_H index
jb_resid, AE
jb_resid, SYM_H index
ms_resid, AE
ms_resid, SYM_H index


C:\Users\murph\AppData\Local\Temp\ipykernel_22888\3350627395.py:37: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [13]:
db_dat.columns

Index(['DateTime', '1300_02', '43000_09', '85550_13', '94400_18',
       'SYM_H index', 'AE', 'SatLat', 'cos_SatMagLT', 'sin_SatMagLT',
       'cos_SatLon', 'sin_SatLon', '400kmDensity', 'storm', 'storm phase',
       '400kmDensity_pred', 'rf_resid', 'dt_resid', 'DTM_400kmDensity',
       'jb_resid', 'JB2008_400kmDensity', 'ms_resid', 'MSIS_400kmDensity'],
      dtype='object')

In [14]:
# k folds parameters
k_folds = 10
k_size = 0.5

# whisker size
whisker = 0.0

# define the x_cols of dat that we want to use for binning
# and their ranges and number of bins
x_dat = ['AE', 'SYM_H index','Den']
x_lab = ['AE (nT)', 'SYM H (nT)','Density (km/m$\mathregular{^3}$ x 10$\mathregular{^{-12}}$)']
x_bin = [20,21,24]
x_range = [[0,2000], [-200,10],[0,12]]
# the y true and predicted values
y_true = ['400kmDensity']
y_pred = ['400kmDensity_pred']
y_pred = ['400kmDensity_pred','DTM_400kmDensity','JB2008_400kmDensity','MSIS_400kmDensity']
y_lab = ['RANDM', 'DTM Op', 'JB2008', 'MSIS']

db_dat['Den'] = db_dat['400kmDensity']

# use accuracy as the metric here
metric = 'Mean Absolute Error'
met = lambda y_true, y_pred: metrics.mean_absolute_error(y_true, y_pred)

In [15]:
showmean=True
fig1, ax1 = plt.subplots(len(x_dat),1, figsize=(6,8))
fig1.set_constrained_layout(True)

# box plot colors
cc = ([1,0,0],
      [0.13333333333333333, 0.5450980392156862, 0.13333333333333333],
      [0.27450980392156865, 0.5098039215686274, 0.7058823529411765],
      [1.0, 0.5490196078431373, 0.0,])
bx_a = 0.25 # transparency level (alpha) for box
ln_a = 1.0 # transparency level for lines
ln_w = 2.0 # line width

for yp, bc, yl in zip(y_pred, cc, y_lab):
    print(yp)
    met_box = inspect.boxplot_metvx(x_dat, y_true, [yp], box_dat=db_dat, bins=x_bin, xrange=x_range,
                                    kfolds=k_folds, kfrac=k_size, box_metric=met, whisker=whisker)



    # boxplot_vx returns a dictionary for each of x_dat which contains 
    # the required values to plot a boxplot using bxp()
    # the dictionary key is the inputs DataFramed column names

    # lets do the same thing but loop through the returned box_dat
    # to create a box/whisker plot for each returned key (x_dat)
    # loop through the dictionary to plot all box_plots
    for (key, value), ap, xl in zip(met_box.items(), ax1, x_lab):
        print(key)
        plt_box = met_box[key]
        y_val = plt_box['box_stats']
        x_val = plt_box['x_centre']
        x_width = plt_box['x_width']
        
        b = ap.bxp(y_val, positions=x_val, widths=x_width, 
                        patch_artist=True, showmeans=showmean, 
                        shownotches=False, showcaps=False, 
                        boxprops={'ec':bc+[ln_a], 'fc':bc+[bx_a]},
                        medianprops={'c':bc, 'lw':ln_w}, 
                        meanprops={'mec':bc, 'mfc':bc}, 
                        label=yl)
        x_tk = np.append(x_val,x_val.max()+x_width)-x_width/2.
        ap.set_xticks(x_tk,x_tk.astype(float).astype(str),rotation=45)
        ap.set_ylabel(metric)
        ap.set_xlabel(xl)

ax1[0].legend(bbox_to_anchor=(1., 1),loc='upper left',fontsize=8)
ax1[0].get_legend().set_title("Model")

ax1[1].legend(bbox_to_anchor=(1., 1),loc='upper left',fontsize=8)
ax1[1].get_legend().set_title("Model")

ax1[2].legend(bbox_to_anchor=(1., 1),loc='upper left',fontsize=8)
ax1[2].get_legend().set_title("Model")

plt.tight_layout()
plt.show()
fig1.savefig('E:/OneDrive/Proposals/NERC_SatDrag/Resid_Func_box.pdf', dpi=300, format='pdf')


400kmDensity_pred
Using passed metric
AE
SYM_H index
Den
DTM_400kmDensity
Using passed metric
AE
SYM_H index
Den
JB2008_400kmDensity
Using passed metric
AE
SYM_H index
Den
MSIS_400kmDensity
Using passed metric
AE
SYM_H index
Den


C:\Users\murph\AppData\Local\Temp\ipykernel_22888\2568308224.py:56: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [16]:
(db_dat['rf_resid']/db_dat['Den']).describe()

count    297756.000000
mean          0.065204
std           8.566724
min       -4618.067678
25%          -0.035193
50%           0.150327
75%           0.290329
max           0.851371
dtype: float64

In [17]:
y_dat = ['rf_resid','dt_resid','jb_resid','ms_resid']
y_lab = ['RF', 'DTM Op', 'JB2008', 'MSIS']

density=False
cumulative = False
hmin=-1
hmax=3

h_bins = np.histogram_bin_edges(db_dat['rf_resid'],bins='fd',range=(hmin,hmax))

whisker = 0 # don't want whiskers
showmean = True # plot the means

# box plot colors
cc = ([1,0,0],
      [0.13333333333333333, 0.5450980392156862, 0.13333333333333333],
      [0.27450980392156865, 0.5098039215686274, 0.7058823529411765],
      [1.0, 0.5490196078431373, 0.0,])


fig3, ax3 = plt.subplots(1,1, figsize=(8,6))

# CHAMP
ax3.hist(db_dat['rf_resid'],bins=h_bins, alpha = 1, label='RF', density=density, color=cc[0], histtype='step', cumulative=cumulative)
ax3.hist(db_dat['dt_resid'],bins=h_bins, alpha = 1, label='DTM Op', density=density, color= cc[1], histtype='step', cumulative=cumulative)
ax3.hist(db_dat['jb_resid'],bins=h_bins, alpha = 0.5, label='JB2008', density=density, color= cc[2], cumulative=cumulative)
ax3.hist(db_dat['ms_resid'],bins=h_bins, alpha = 0.5, label='MSIS', density=density, color= cc[3], cumulative=cumulative)
ax3.set(title='OOS - CHAMP, Quiet Times', xlabel='Residuals (Obs-Pred)', ylabel='Probability')

ax3.legend(bbox_to_anchor=(1., 1),loc='upper right',fontsize=8)
ax3.get_legend().set_title("Model")
ax3.grid(axis='both')


In [18]:
# large storm
sdate = '2005-08-22'
edate = '2005-08-30'

In [19]:
db_dat.columns

Index(['DateTime', '1300_02', '43000_09', '85550_13', '94400_18',
       'SYM_H index', 'AE', 'SatLat', 'cos_SatMagLT', 'sin_SatMagLT',
       'cos_SatLon', 'sin_SatLon', '400kmDensity', 'storm', 'storm phase',
       '400kmDensity_pred', 'rf_resid', 'dt_resid', 'DTM_400kmDensity',
       'jb_resid', 'JB2008_400kmDensity', 'ms_resid', 'MSIS_400kmDensity',
       'Den'],
      dtype='object')

In [20]:
st_t = (db_dat['DateTime'] > sdate) & (db_dat['DateTime'] <= edate)

In [21]:
y_true = '400kmDensity'
y_pred = '400kmDensity_pred'
on = 'DateTime'
rkwargs = {'window':'90min','center':True}
met = {'MAE':lambda y_true, y_pred: metrics.mean_absolute_error(y_true, y_pred)}

In [22]:
r_met = inspect.rolling_met(db_dat[st_t],y_true=y_true,y_pred=y_pred,on=on,
                        roll_kwargs=rkwargs, roll_metric=met)

Using passed metric


In [23]:
y_pred = ['400kmDensity_pred','DTM_400kmDensity','JB2008_400kmDensity','MSIS_400kmDensity'] 
for y in y_pred:
    r_met[f'MAE {y}'] = inspect.rolling_met(db_dat[st_t],y_true=y_true,y_pred=y,on=on,
                        roll_kwargs=rkwargs, roll_metric=met)['MAE']

Using passed metric
Using passed metric
Using passed metric
Using passed metric


In [24]:
r_met.columns

Index(['MAE', 'DateTime', 'MAE 400kmDensity_pred', 'MAE DTM_400kmDensity',
       'MAE JB2008_400kmDensity', 'MAE MSIS_400kmDensity'],
      dtype='object')

In [25]:
sfig = True
plt.rcParams.update({'font.size': 9})
plt.subplots_adjust(hspace=0)
#fig, ax = plt.subplots(8,1,figsize=(5,8),sharex=True, layout='constrained')
fig1, ax1 = plt.subplots(5,1,figsize=(5,6),sharex=True, layout='constrained')


db_dat[st_t].plot(x='DateTime', y='AE', ylabel='AE', ax=ax1[0], legend=False)
db_dat[st_t].plot(x='DateTime', y='SYM_H index', ylabel='Sym-H', ax=ax1[1], legend=False)

db_dat[st_t].plot(x='DateTime', y='400kmDensity', ylabel='CHAMP Density', ax=ax1[2], legend=False)

plt_par = ['400kmDensity','400kmDensity_pred','DTM_400kmDensity','JB2008_400kmDensity','MSIS_400kmDensity'] 

db_dat[st_t].rolling('90min',on='DateTime').mean().\
plot(x='DateTime', y=plt_par, xlim=[sdate,edate], ax=ax1[3], 
     label=['Obs','RF','DTM Op', 'JB2008', 'MSIS'], ylabel='Density', legend=True, ylim=[0,7])

r_met.plot(x='DateTime', y=['MAE 400kmDensity_pred', 'MAE DTM_400kmDensity','MAE JB2008_400kmDensity', 'MAE MSIS_400kmDensity'],
           ax=ax1[4], label=['RF','DTM Op', 'JB2008', 'MSIS'], ylabel='MEA', legend=True, ylim=[0,3])



<Axes: xlabel='DateTime', ylabel='MEA'>

In [26]:
ss = '2005-08-24 06:00:00'
ee = '2005-08-25 06:00:00'

d_t = (db_dat['DateTime'] > ss) & (db_dat['DateTime'] <= ee)
r_st = (r_met['DateTime'] > ss) & (r_met['DateTime'] <= ee)

sfig = True
plt.rcParams.update({'font.size': 9})
plt.subplots_adjust(hspace=0)
fig1, ax2 = plt.subplots(2,1,figsize=(3,4),sharex=True, layout='constrained')

db_dat[d_t].rolling('90min',on='DateTime').mean().\
plot(x='DateTime', y=plt_par, xlim=[ss,ee], ax=ax2[0], label=['Obs','RF','DTM Op', 'JB2008', 'MSIS'], ylabel='Density', legend=True)

r_met[r_st].plot(x='DateTime', y=['MAE 400kmDensity_pred', 'MAE DTM_400kmDensity','MAE JB2008_400kmDensity', 'MAE MSIS_400kmDensity'],
           ax=ax2[1], label=['RF','DTM Op', 'JB2008', 'MSIS'], ylabel='MEA', legend=True, ylim=[0,11], xlim=[ss,ee])



C:\Users\murph\AppData\Local\Temp\ipykernel_22888\2149014880.py:9: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  plt.subplots_adjust(hspace=0)


<Axes: xlabel='DateTime', ylabel='MEA'>